## Funciones de utilidad

In [ ]:
import sympy as sp
from IPython.display import display, Math

def print_math(sympy_exp, pre_symbol = '', post_symbol = '', convert_pre_post = False):
    # Map some symbols to be reaplaced before printing to keep desired order
    rep_map = {
        'zt': 't',
        'zU_p': 'U_p',
        'z\\dot{U}_p': '\\dot{U}_p',
        'z\\ddot{U}_p': '\\ddot{U}_p',
        'zU': 'U',
        'z\\dot{U}': '\\dot{U}',
        'z\\ddot{U}': '\\ddot{U}',
        'a\\xi': '\\xi',
        'am': 'm',
        'bc': 'c',
    }
    latex_str = sp.latex(sympy_exp)
    if convert_pre_post:
        pre_symbol_latex = sp.latex(pre_symbol)
        post_symbol_latex = sp.latex(post_symbol)
    for old_str in rep_map:
        latex_str = str(latex_str).replace(old_str, rep_map[old_str])
        if pre_symbol != '' and convert_pre_post:
            pre_symbol_latex = pre_symbol_latex.replace(old_str, rep_map[old_str])
        if post_symbol != '' and convert_pre_post:
            post_symbol_latex = post_symbol_latex.replace(old_str, rep_map[old_str])
            
    # if pre_symbol != '' and convert_pre_post:
    #     pre_symbol = pre_symbol_latex
    # if post_symbol != '' and convert_pre_post:
    #     post_symbol = post_symbol_latex
        
    if pre_symbol != '':
        if convert_pre_post:
            pre_symbol = pre_symbol_latex
        latex_str = pre_symbol + ' = ' + latex_str
    if post_symbol != '':
        if convert_pre_post:
            post_symbol = post_symbol_latex
        latex_str = latex_str + ' = ' + post_symbol
    # print(latex_str)
    display(Math(f'{latex_str} \n'))

def print_text(text_str):
    display(Math(f'\\textrm{{{text_str}}}'))

## Solución homogenea para no forzado!

In [ ]:
# Solution for the homogeneus part
Uu, Uv, Ua, A, B, wn, wd, t, m, k, to, z, c = sp.symbols('zU, z\\dot{U}, z\\ddot{U}, A, B, w_n, w_d, zt, am, k, zt_0, a\\xi, bc')

Uuo, Uvo, Uao = sp.symbols('zU_0, \\dot{U}_0, \\ddot{U}_0')

# ----------- Non-damped free vibration -----------
# Diferential equation
# print_text('Governing differential equation: ')
# ed = m * Ua + k * Uu
# print_math(ed, '', '0')
# print_text('Solution shape: ')
# Us = A * sp.cos(wn * t) + B * sp.sin(wn * t)
# print_math(Us, 'U')
# Vs = sp.diff(Us, t)
# print_math(Vs, '\\dot{U}')
# As = sp.diff(Vs, t)
# print_math(As, '\\ddot{U}')

# ----------- Damped free vibration -----------
# Diferential equation
print_text('Governing differential equation: ')
ed = m * Ua + c * Uv + k * Uu
print_math(ed, '', '0')
print_text('Solution shape: ')
Us = (A * sp.cos(wd * t) + B * sp.sin(wd * t)) * sp.exp(-z*wn*t)
print_math(Us, 'U')
Vs = sp.diff(Us, t)
print_math(Vs, '\\dot{U}')
As = sp.diff(Vs, t)
print_math(As, '\\ddot{U}')

print_text('Given the boundary conditions, find the coefficients: ')
Uso = Us - Uuo
Uso = Uso.subs(t, to)
print_math(Uso, '', '0')
Uso = Uso.subs(to, 0)
Vso = Vs - Uvo
Vso = Vso.subs(t, to)
print_math(Vso, '', '0')
Vso = Vso.subs(to, 0)

print_math(Uso, '', '0')
print_math(Vso, '', '0')

coeff_sol = sp.solve([Uso, Vso], [A, B])
for coeff in coeff_sol:
    print_math(coeff_sol[coeff], sp.latex(coeff))

print_text('Final solutions:')
Uu = Us.subs(coeff_sol)
print_math(Uu, 'U')
Uv = Vs.subs(coeff_sol)
print_math(Uv, '\\dot{U}')
Ua = As.subs(coeff_sol)
print_math(Ua, '\\ddot{U}')


## Encontrando la solución particular

In [52]:
# General symbols
Uu, Uv, Ua, A, B, wn, wd, t, m, k, to, z, c, r = sp.symbols('zU, z\\dot{U}, z\\ddot{U}, A, B, w_n, w_d, zt, am, k, zt_0, a\\xi, bc, r')

# Symbols for the particular solution
Uup, Uvp, Uap, Ap, Bp, Uest = sp.symbols('zU_p, z\\dot{U}_p, z\\ddot{U}_p, A_p, B_p, U_{est}')

# Symbols specific for the loading function
Ome, Fo = sp.symbols('\Omega, F_0')


# Parts of the particular solution
part_factors = [sp.cos(Ome * t), sp.sin(Ome * t)]

# External force function
ext_force = Fo * sp.sin(Ome * t)

# Diferential equation
print_text('Governing differential equation: ')
ed = m * Ua + c * Uv + k * Uu # - Fo * sp.sin(Ome * t)
print_math(ed, '', ext_force, True)

# TODO Divide by m to make things easier and get \xi and wn into play
ed = sp.expand(ed / m)
ed = ed.subs(c, 2*z*wn*m)
ed = ed.subs(k/m, wn**2)
ext_force = ext_force / m
print_math(ed, '', ext_force, True)

# Solution's shape for displacement, velocity and acceleration
print_text('Solution shape: ')
Ups = (Ap * part_factors[0] + Bp * part_factors[1])
print_math(Ups, 'U_p',)
Vps = sp.diff(Ups, t)
print_math(Vps, '\\dot{U_p}')
Aps = sp.diff(Vps, t)
print_math(Aps, '\\ddot{U_p}')

# Plug solution function into the governing differential equation
ed_rep = ed.subs({Uu: Ups, Uv: Vps, Ua: Aps})
print_math(ed_rep, '', ext_force, True)

# TODO Introduce the factor r = Ome/wn
ed_rep = sp.expand(ed_rep / wn**2)
ed_rep = ed_rep.subs(Ome / wn, r)
ext_force = sp.expand(ext_force / wn**2)
ext_force = ext_force.subs(m * wn**2, k)
ext_force = ext_force.subs(Fo / k, Uest)
print_math(ed_rep, '', ext_force, True)

# Get the factors for each part of the function to solve it
factors_coeff = sp.reduced(ed_rep, part_factors)
factors_coeff_ext = sp.reduced(ext_force, part_factors)

# Create the system of equations
eqns = []
for i in range(len(factors_coeff[0])):
    eqns.append(factors_coeff[0][i] - factors_coeff_ext[0][i])

# Solve the system of equations
sol = sp.solve(eqns, [Ap, Bp])

print_text('Coeficientes encontrados: ')
for res in sol:
    sol[res] = sp.simplify(sp.factor(sol[res]))
    print_math(sol[res], res, '', True)

print_text('Solución particular: ')
# Plug found coefficients into the governing equation
Ups = sp.simplify(Ups.subs(sol))
print_math(Ups, 'U_p',)
Vps = sp.simplify(Vps.subs(sol))
print_math(Vps, '\\dot{U}_p',)
Aps = sp.simplify(Aps.subs(sol))
print_math(Aps, '\\ddot{U}_p',)

# print_text('Given the boundary conditions, find the coefficients: ')
# Uso = Us - Uuo
# Uso = Uso.subs(t, to)
# print_math(Uso, '', '0')
# Uso = Uso.subs(to, 0)
# Vso = Vs - Uvo
# Vso = Vso.subs(t, to)
# print_math(Vso, '', '0')
# Vso = Vso.subs(to, 0)

# print_math(Uso, '', '0')
# print_math(Vso, '', '0')

# coeff_sol = sp.solve([Uso, Vso], [A, B])
# for coeff in coeff_sol:
#     print_math(coeff_sol[coeff], sp.latex(coeff))

# print_text('Final solutions:')
# Uu = Us.subs(coeff_sol)
# print_math(Uu, 'U')
# Uv = Vs.subs(coeff_sol)
# print_math(Uv, '\\dot{U}')
# Ua = As.subs(coeff_sol)
# print_math(Ua, '\\ddot{U}')


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Solución total con $U = U_p + U_h$

In [53]:
Uuo, Uvo, Uao = sp.symbols('zU_0, \\dot{U}_0, \\ddot{U}_0')

# ----------- Damped free vibration -----------
# Diferential equation
print_text('Governing differential equation: ')
ed = m * Ua + c * Uv + k * Uu
print_math(ed, '', '0')
print_text('Solution shape: ')
Us = (A * sp.cos(wd * t) + B * sp.sin(wd * t)) * sp.exp(-z*wn*t) + Ups
print_math(Us, 'U')
Vs = sp.diff(Us, t)
print_math(Vs, '\\dot{U}')
As = sp.diff(Vs, t)
print_math(As, '\\ddot{U}')

print_text('Given the boundary conditions, find the coefficients: ')
Uso = Us - Uuo
Uso = Uso.subs(t, to)
print_math(Uso, '', '0')
Uso = Uso.subs(to, 0)
Vso = Vs - Uvo
Vso = Vso.subs(t, to)
print_math(Vso, '', '0')
Vso = Vso.subs(to, 0)

print_text('System of equations to solve: ')
print_math(Uso, '', '0')
print_math(Vso, '', '0')


print_text('Coeficientes encontrados: ')
coeff_sol = sp.solve([Uso, Vso], [A, B])
for coeff in coeff_sol:
    coeff_sol[coeff] = sp.simplify(sp.factor(coeff_sol[coeff]))
    print_math(coeff_sol[coeff], sp.latex(coeff))

print_text('Final solutions:')
Uu = sp.simplify(Us.subs(coeff_sol))
print_math(Uu, 'U')
Uv = sp.simplify(Vs.subs(coeff_sol))
print_math(Uv, '\\dot{U}')
Ua = sp.simplify(As.subs(coeff_sol))
print_math(Ua, '\\ddot{U}')


<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

## Translating the final solutions to Matlab

In [72]:
U_str = str(Uu)
V_str = sp.latex(Uv)
A_str = sp.latex(Ua)

U_str = U_str.replace('**', '^')
used_symbols =  Uu.atoms(sp.Symbol)

rep_map = {
    'w_n': 'wn',
    '\\Omega' : 'Ome',
    'zU_0' : 'Uo',
    'r' : 'r',
    'w_d' : 'wd',
    'zt' : 't',
    'U_{est}' : 'Uest',
    '\\dot{U}_0' : 'Vo',
    'a\\xi' : 'z',
}

print('Original: ')
print(U_str)

for sym in used_symbols:
    U_str = U_str.replace(str(sym), rep_map[str(sym)])

print('Matlab expression: ')
print(U_str)


Original: 
(-U_{est}*w_d*(2*a\xi*r*cos(\Omega*zt) + (r^2 - 1)*sin(\Omega*zt))*exp(a\xi*w_n*zt) + w_d*(2*U_{est}*a\xi*r + 4*a\xi^2*r^2*zU_0 + r^4*zU_0 - 2*r^2*zU_0 + zU_0)*cos(w_d*zt) + (U_{est}*\Omega*r^2 - U_{est}*\Omega + 2*U_{est}*a\xi^2*r*w_n + 4*\dot{U}_0*a\xi^2*r^2 + \dot{U}_0*r^4 - 2*\dot{U}_0*r^2 + \dot{U}_0 + 4*a\xi^3*r^2*w_n*zU_0 + a\xi*r^4*w_n*zU_0 - 2*a\xi*r^2*w_n*zU_0 + a\xi*w_n*zU_0)*sin(w_d*zt))*exp(-a\xi*w_n*zt)/(w_d*(4*a\xi^2*r^2 + r^4 - 2*r^2 + 1))
Matlab expression: 
(-Uest*wd*(2*z*r*cos(Ome*t) + (r^2 - 1)*sin(Ome*t))*exp(z*wn*t) + wd*(2*Uest*z*r + 4*z^2*r^2*Uo + r^4*Uo - 2*r^2*Uo + Uo)*cos(wd*t) + (Uest*Ome*r^2 - Uest*Ome + 2*Uest*z^2*r*wn + 4*Vo*z^2*r^2 + Vo*r^4 - 2*Vo*r^2 + Vo + 4*z^3*r^2*wn*Uo + z*r^4*wn*Uo - 2*z*r^2*wn*Uo + z*wn*Uo)*sin(wd*t))*exp(-z*wn*t)/(wd*(4*z^2*r^2 + r^4 - 2*r^2 + 1))
